In [ ]:
import dask.dataframe as dd
import pandas as pd

import panel as pn
pn.extension("perspective")
pn.config.sizing_mode="stretch_width"

import holoviews as hv
hv.extension("bokeh")
import hvplot.dask 
import hvplot.pandas
import numpy as np

from config import TRAINING_GOLD_PATH, MODEL_PATH
import pickle

## Train Simple Load Model

Feel free to improve :-)

In [ ]:
data = dd.read_parquet(TRAINING_GOLD_PATH).compute().set_index("datetime")
data.head()

In [ ]:
data.plot(y="load", figsize=(25,10))

In [ ]:
kdims=["temperature", "dewpoint", ] + ["load"]
data = data[kdims].dropna()
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

In [ ]:
# evaluate an lasso regression model on the dataset
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
# load the dataset
X, y = data[["temperature", "dewpoint"]], data["load"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# define model
model = Lasso(alpha=1.0)
result=model.fit(X_train, y_train)
y_predict = model.predict(X_test)

## Test Load Model

In [ ]:
score = result.score(X_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))

In [ ]:
df_test=pd.concat([X_test, y_test], axis=1)
df_test["load_predict"]=y_predict
df_test = df_test.sort_index().reset_index()

In [ ]:
# df_test.hvplot(x="datetime", y=["load", "load_predict"], datashade=True, color="yellow" )
df_test.plot(x="datetime", y=["load", "load_predict"], figsize=(25,10))

## Save Load Model

In [ ]:
with open(MODEL_PATH, 'wb') as file:
    pickle.dump(result, file)

## Predict Using Load Model

In [ ]:
temperature=10
dewpoint=8
result.predict([[temperature, dewpoint]])[0].round(0)

Previous: [Viz Training Data Set](viz_training_dataset.ipynb)